
## [2017-01-27 Workshop: Data Science with Maps | Meetup Page](https://www.meetup.com/CartoCamp/events/236668763/)

### Get your Carto API
[http://bit.ly/2jmWj5e](http://bit.ly/2jmWj5e)

### Create a Virtual Environment
If you don't have `virtualenv`

    pip install virtualenv

Create a folder in the current directory which will contain the Python executable files, and a copy of the pip library which you can use to install other packages. 

	virtualenv env

Activate Virtual Env.

	source env/bin/activate
	
Install Pandas OR maybe add REQUIREMENTS.txt
    
    pip install jupyter
    pip install pandas
	pip install requests

### Our Data, NYC Taxi Data

http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml

    
### Update and create a credentials.json modeled after credentials.json.sample

    {"username": "eschbacher",
     "api_key": "abcdefghijklmnopqrstuvwxyz1234567890"}
     

In [65]:
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 100
pd.options.display.max_rows    = 100
pd.set_option('display.width', 100)    #http://stackoverflow.com/questions/11707586/python-pandas-widen-output-display
pd.set_option('display.max_rows', 500) #http://stackoverflow.com/questions/11707586/python-pandas-widen-output-display

# Getting a Dataframe from Carto's SQL API 

In [66]:
import json
import pandas as pd 
import requests
import io
import os
import shutil

cred = json.load(open('credentials_nygeog.json')) # modify credentials.json.sample
source_username = 'sheehan-carto' #leave this as is, the table we're pulling from is in my account
username = cred['username']
api_key  = cred['api_key']

tablename = 'yellow_tripdata_2015_04_15_n_3000'
size      = 3000

url = 'https://%s.carto.com/api/v2/sql?q=SELECT * FROM %s LIMIT %s&format=csv' % (source_username , tablename, str(size))

s  = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

print df.head(3)

   cartodb_id  the_geom  the_geom_webmercator  vendorid    tpep_pickup_datetime  \
0           1       NaN                   NaN         1  2015-04-15 19:35:10+00   
1           2       NaN                   NaN         1  2015-04-15 19:35:10+00   
2           3       NaN                   NaN         1  2015-04-15 19:35:11+00   

    tpep_dropoff_datetime  passenger_count  trip_distance  pickup_longitude  pickup_latitude  \
0  2015-04-15 20:22:59+00                1           13.2        -73.982346        40.773956   
1  2015-04-15 19:44:02+00                1            2.1        -73.987564        40.755409   
2  2015-04-15 19:41:43+00                1            0.9          0.000000         0.000000   

   ratecodeid store_and_fwd_flag  dropoff_longitude  dropoff_latitude  payment_type  fare_amount  \
0           1              False         -73.946808         40.631737             1         44.0   
1           1              False         -74.005234         40.728886             

# Get columns for desired output and intersect with tracts

In [67]:
cols = []

for i in df.columns:
	cols.append(i)

del cols[:3] #delete cartodb_id, the_geom, the_geom_webmercator,
cols = cols + ['geoid']    
c = ', '.join(cols) #list of cols to keep to string for SQL statement
    
points = 'yellow_tripdata_2015_04_15_n_3000'
poly   = 'tl_2014_census_tracts_statefp_36'    
lat    = points + '.pickup_latitude'
lng    = points + '.pickup_longitude'

url = "https://%s.carto.com/api/v2/sql?q=SELECT %s FROM %s, %s  WHERE ST_INTERSECTS(ST_SetSRID(ST_Point(%s, %s),4326), %s.the_geom)&format=csv" % (source_username, c, points, poly, lng, lat, poly )
s  = requests.get(url).content

df = pd.read_csv(io.StringIO(s.decode('utf-8')))

try:
    df = df.rename(columns=lambda x: x.replace('geoid', 'pickup_geoid'))
except:
    raise
    
dfp = df[['pickup_geoid']]
dfp['count_pickup'] = 1
dfp = dfp.groupby(['pickup_geoid'],as_index=False).sum()

print dfp.head(3)

if not os.path.exists('temp'): #create temp folder
    os.makedirs('temp')

print len(dfp.index)

dfp.to_csv('temp/'+points+'.csv', index=False)

   pickup_geoid  count_pickup
0   36005009300             1
1   36005014100             1
2   36047000502             1
293


# Send your .csv up to Carto

In [70]:
os.system('curl -v -F file=@temp/'+points+'.csv "https://'+username+'.carto.com/api/v1/imports/?api_key="'+api_key)

try:
    shutil.rmtree('temp') #delete temp folder
except:
    print 'temp folder not there'

temp folder not there


# Using SQL JOIN Pickup Counts to Tracts

### First Copy these files into you Carto Account

1. COPY AND PASTE THIS HERE, https://team.carto.com/u/sheehan-carto/dashboard CONNECT DATASET

    https://sheehan-carto.carto.com:443/api/v2/sql?q=select%20*%20from%20"sheehan-carto".tl_2014_census_tracts_statefp_36&format=geojson&filename=tl_2014_census_tracts_statefp_36
    
 
2. COPY AND PASTE THIS HERE, https://team.carto.com/u/sheehan-carto/dashboard CONNECT DATASET
 
    https://sheehan-carto.carto.com:443/api/v2/sql?q=select%20*%20from%20"sheehan-carto".yellow_tripdata_2015_04_15_n_3000&format=geojson&filename=yellow_tripdata_2015_04_15_n_3000

In [73]:
new_table = 'yellow_pickups_by_tract_2015_04_15_n_3000'
         
url = "https://%s.carto.com/api/v2/sql?q=CREATE TABLE %s AS SELECT %s.the_geom, %s.geoid , %s.count_pickup FROM %s, %s WHERE %s.geoid = %s.pickup_geoid::text&api_key=%s" % (username, new_table, poly, poly, points, poly, points, poly, points, api_key)
print url
s  = requests.get(url).content

https://nygeog.carto.com/api/v2/sql?q=CREATE TABLE yellow_pickups_by_tract_2015_04_15_n_3000 AS SELECT tl_2014_census_tracts_statefp_36.the_geom, tl_2014_census_tracts_statefp_36.geoid , yellow_tripdata_2015_04_15_n_3000.count_pickup FROM tl_2014_census_tracts_statefp_36, yellow_tripdata_2015_04_15_n_3000 WHERE tl_2014_census_tracts_statefp_36.geoid = yellow_tripdata_2015_04_15_n_3000.pickup_geoid::text&api_key=4afbc252f49ea8a83920e2378df77fbe0a58a806


# CartoDBFY Your New Table 
    SELECT cdb_cartodbfytable('your-account-name', 'your-table-name');
OR 
    
    SELECT cdb_cartodbfytable('your-table-name');

In [63]:
If you have a team CARTO account, run this, commment this out
url = "https://%s.carto.com/api/v2/sql?q=SELECT cdb_cartodbfytable('%s','%s')&api_key=%s" % (username, username, new_table, api_key)
print url      
s  = requests.get(url).content

https://sheehan-carto.carto.com/api/v2/sql?q=SELECT cdb_cartodbfytable('sheehan-carto','yellow_pickups_by_tract_2015_04_15_n_3000')&api_key=6f74410ee81d7db76ccd338768a6790910a0c920


In [74]:
#If you have a free or basic or personal CARTO account, run this, commment this out
url = "https://%s.carto.com/api/v2/sql?q=SELECT cdb_cartodbfytable('%s')&api_key=%s" % (username, new_table, api_key)
print url 
s  = requests.get(url).content

https://nygeog.carto.com/api/v2/sql?q=SELECT cdb_cartodbfytable('yellow_pickups_by_tract_2015_04_15_n_3000')&api_key=4afbc252f49ea8a83920e2378df77fbe0a58a806
